In [3]:
#importing of the required modules
import cv2
from collections import Counter
from bitarray import bitarray
import math
import matplotlib.pyplot as plt
import huffman
import queue
from heapq import heappush, heappop

In [4]:

# Load grayscale image
lhs = cv2.imread('lhs.png', cv2.IMREAD_GRAYSCALE)
rhs = cv2.imread('rhs.png', cv2.IMREAD_GRAYSCALE)



In [5]:

# Convert image to a sequence of symbols (pixel values)
symbolsLHS = lhs.flatten()
print(symbolsLHS)
symbolsRHS = rhs.flatten()
print(symbolsRHS)


# Calculate symbol frequencies
freqLHS = Counter(symbolsLHS)
print(freqLHS)
freqRHS = Counter(symbolsRHS)
print(freqRHS)


[197 193 187 ... 112 102  98]
[193 189 182 ... 111 103  94]
Counter({21: 54706, 0: 46963, 28: 33690, 34: 26286, 39: 21098, 43: 19385, 46: 19119, 50: 14866, 53: 11685, 56: 9828, 255: 8834, 59: 8022, 61: 7247, 64: 6425, 156: 6074, 66: 5878, 160: 5689, 151: 5510, 164: 5412, 68: 5199, 144: 4770, 170: 4754, 70: 4753, 199: 4725, 167: 4722, 173: 4678, 72: 4614, 186: 4589, 188: 4553, 180: 4549, 197: 4526, 205: 4524, 194: 4515, 213: 4503, 254: 4495, 192: 4490, 206: 4475, 184: 4474, 200: 4474, 209: 4473, 202: 4469, 195: 4441, 182: 4402, 203: 4388, 175: 4387, 190: 4347, 217: 4329, 178: 4306, 212: 4223, 210: 4222, 219: 4210, 218: 4188, 74: 4178, 207: 4171, 76: 4128, 214: 4006, 215: 3957, 220: 3759, 78: 3750, 221: 3619, 80: 3611, 82: 3467, 225: 3450, 224: 3378, 84: 3349, 223: 3312, 157: 3292, 85: 3249, 159: 3246, 158: 3240, 162: 3229, 226: 3163, 87: 3154, 89: 3100, 161: 3024, 90: 3006, 154: 2975, 152: 2960, 153: 2931, 163: 2889, 165: 2878, 148: 2874, 149: 2855, 93: 2801, 155: 2781, 92: 2779, 150: 2

In [7]:


# Build Tree

class Node:
    def __init__(self, symbol=None, frequency=0, left=None, right=None):
        self.symbol = symbol
        self.frequency = frequency
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.frequency < other.frequency

    def __eq__(self, other):
        return self.frequency == other.frequency

def generate_huffman_tree(frequencies):
    # Create list of nodes
    nodes = [Node(symbol, frequency) for symbol, frequency in frequencies.items()]

    # Build Huffman tree iteratively
    while len(nodes) > 1:
        # Pop two nodes with smallest frequencies
        node1 = heappop(nodes)
        node2 = heappop(nodes)

        # Create parent node with combined frequency
        parent = Node(frequency=node1.frequency + node2.frequency, left=node1, right=node2)

        # Push parent node back onto heap
        heappush(nodes, parent)

    # Return root node of Huffman tree
    return nodes[0]

treeLHS = generate_huffman_tree(freqLHS)
print(treeLHS)
treeRHS = generate_huffman_tree(freqRHS)
print(treeRHS)


In [8]:

# Generate binary codes for each symbol
def generate_huffman_table(huffman_tree):
    codes = {}

    def traverse(node, code):
        if node.symbol is not None:
            codes[node.symbol] = code
        else:
            traverse(node.left, code + "0")
            traverse(node.right, code + "1")

    traverse(huffman_tree, "")
    return codes

tableLHS = generate_huffman_table(treeLHS)
tableRHS = generate_huffman_table(treeRHS)

# Encode symbols using binary codes
encodedLHS = bitarray()
for symbol in symbolsLHS:
    encodedLHS.extend(tableLHS[symbol])
encodedRHS = bitarray()
for symbol in symbolsRHS:
    encodedRHS.extend(tableRHS[symbol])


In [9]:

# Calculate number of bits required to represent image using Huffman coding
num_bits_huffmanLHS = len(encodedLHS)
num_bits_huffmanRHS = len(encodedRHS)

# Calculate entropy of source sequence
nLHS = len(symbolsLHS)
probabilitiesLHS = [float(freqLHS[symbol]) / nLHS for symbol in freqLHS]
entropyLHS = -sum(p * math.log2(p) for p in probabilitiesLHS)
nRHS = len(symbolsRHS)
probabilitiesRHS = [float(freqRHS[symbol]) / nRHS for symbol in freqRHS]
entropyRHS = -sum(p * math.log2(p) for p in probabilitiesRHS)

# Print results
print("Number of bits using Huffman coding for LHS:", num_bits_huffmanLHS)
print("Entropy of source sequence for LHS:", entropyLHS)

print("Number of bits using Huffman coding for RHS:", num_bits_huffmanRHS)
print("Entropy of source sequence for RHS:", entropyRHS)

Number of bits using Huffman coding for LHS: 5469938
Entropy of source sequence for LHS: 6.862043223064135
Number of bits using Huffman coding for RHS: 6227919
Entropy of source sequence for RHS: 7.859514285787118
